# Detección de cambios en los manuscritos

## Carga de datos
Cargamos los datos del histórico db_editorial_manager.csv y del scrap n_proceso_actual.csv y definitiva_actual.csv

In [1]:
import pandas as pd

#get the old data from db_editorial_manager.csv and store it in a dataframe called old_data
old_data = pd.read_csv('db_editorial_manager.csv')

last_row = old_data.index[-1]

#filter the rows that have the column fecha_baja == null
old_data = old_data[old_data['fecha_baja'].isnull()]

#old_data.head()
old_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 126 entries, 14 to 764
Data columns (total 63 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   data_identity               0 non-null      float64
 1   numero_manuscrito           126 non-null    object 
 2   referencia                  126 non-null    object 
 3   revision                    0 non-null      float64
 4   titulo                      126 non-null    object 
 5   tipo_articulo               0 non-null      float64
 6   autor                       0 non-null      float64
 7   email                       0 non-null      float64
 8   fecha_inicial               0 non-null      float64
 9   estado_actual               0 non-null      float64
 10  fecha_estado                0 non-null      float64
 11  fecha_vencimiento           0 non-null      float64
 12  editor                      0 non-null      float64
 13  decision_editor_def         0 non-

In [2]:
#get the new data from en_proceso_actual.csv and definitiva_actual.csv and store it in two dataframes called en_procoeso and definitiva
en_proceso = pd.read_csv('scrap/en_proceso_actual.csv')
definitiva = pd.read_csv('scrap/definitiva_actual.csv')

print (en_proceso.info())
print (definitiva.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   numero_manuscrito  69 non-null     object 
 1   tipo_articulo      69 non-null     object 
 2   titulo             69 non-null     object 
 3   autor              69 non-null     object 
 4   fecha_inicial      69 non-null     object 
 5   fecha_estado       69 non-null     object 
 6   estado_actual      69 non-null     object 
 7   editor             29 non-null     object 
 8   decision_editor    48 non-null     object 
 9   est_ev_completo    29 non-null     float64
 10  est_ev_aceptado    29 non-null     float64
 11  est_ev_invitado    29 non-null     float64
 12  est_ev_rechazado   29 non-null     float64
 13  est_ev_tarde       29 non-null     float64
 14  data_identity      69 non-null     int64  
 15  email              40 non-null     object 
 16  fecha_vencimiento  40 non-nu

## Nuevo manuscrito

Al scrapear la página vemos que hay artículos que no están en bd_editorial_manager.csv pero sí en en_proceso_actual.csv

In [3]:
#find new manuscritos in en_proceso that are not in old_data comparing the first 16 characters of numero_manuscrito column
new_manuscritos_en_proceso = en_proceso[~en_proceso.numero_manuscrito.str.contains('|'.join(old_data.numero_manuscrito.str[:16]))]

print (len(new_manuscritos_en_proceso), 'manuscritos nuevos en en_proceso')
#print numero_manuscrito  and estado_actual
print (new_manuscritos_en_proceso[['numero_manuscrito', 'estado_actual']])


4 manuscritos nuevos en en_proceso
   numero_manuscrito         estado_actual
0   NEFRO-D-23-00040  Pendiente del Editor
1   NEFRO-D-23-00039  Pendiente del Editor
34  NEFRO-D-23-00013  Revise / A Modificar
35  NEFRO-D-23-00012  Revise / A Modificar


In [4]:
#we need the same columns in the new dataframes as in the old one.
#we will add the columns that are missing in the new dataframe
import numpy as np

#columns that are in old_data but not in new_manuscritos_en_proceso
columns_to_add = set(old_data.columns).difference(new_manuscritos_en_proceso.columns)

#add the missing columns to new_manuscritos_en_proceso and fill them with NaN
new_manuscritos_en_proceso = new_manuscritos_en_proceso.assign(**{col: np.nan for col in columns_to_add})

In [5]:
#rename 'decision_editor' to 'decision_editor_def' in new_manuscritos_en_proceso

new_manuscritos_en_proceso['decision_editor_def'] = new_manuscritos_en_proceso['decision_editor'].copy()
new_manuscritos_en_proceso.drop(columns=['decision_editor'], inplace=True)


In [6]:
#calculate today date in dd/mm/yyyy format
import datetime 
today = datetime.datetime.now().strftime("%d/%m/%Y")
print (today)

31/01/2023


In [7]:
#calculated fields in old_data
    # referencia -> numero_manuscrito (first 16 characters)
    # revision -> numero_manuscrito (characters 17-18)
    # año -> numero_manuscrito (characters 9-10 + 2000)
    # fecha_alta -> today

#calculated fields in new_manuscritos_en_proceso
for index, row in new_manuscritos_en_proceso.iterrows():
    # referencia -> referencia (first 16 characters)
    new_manuscritos_en_proceso.loc[index, 'referencia'] = row['numero_manuscrito'][:16]
    # revision -> referencia (characters 17-18)
    new_manuscritos_en_proceso.loc[index, 'revision'] = row['numero_manuscrito'][16:18]
    # año -> referencia (characters 9-10 + 2000)
    new_manuscritos_en_proceso.at[index, 'año'] = int(row['numero_manuscrito'][8:10]) + 2000
    # fecha_alta -> today
    new_manuscritos_en_proceso.at[index, 'fecha_alta'] = today

In [8]:

#fill the calculated fields in new_manuscritos_en_proceso
for index, row in new_manuscritos_en_proceso.iterrows():
    # if estado_actual == Pendiente del Editor
        # fecha_estado -> entrada
        # estado_actual	-> 1. Nuevo esperando revisor o rechazar
        #responsable -> Mariano / Roberto
    if row['estado_actual'] == 'Pendiente del Editor':
        new_manuscritos_en_proceso.at[index, 'entrada'] = row['fecha_estado']
        new_manuscritos_en_proceso.at[index, 'estado'] = '1. Nuevo esperando revisor o rechazar'
        new_manuscritos_en_proceso.at[index, 'responsable'] = 'Mariano / Roberto'

    #if estado_actual == Revise / A Modificar
        # fecha_estado -> entrada
        # fecha_estado -> decision_editor_r1
    elif row['estado_actual'] == 'Revise / A Modificar':
        new_manuscritos_en_proceso.at[index, 'entrada'] = row['fecha_estado']
        new_manuscritos_en_proceso.at[index, 'decision_editor_r1'] = row['fecha_estado']

        # if decision_editor == Grandes cambios / Major revision
            # estado -> 4.2 Grandes cambios
        if row['decision_editor_def'] == 'Grandes cambios / Major revision':
            new_manuscritos_en_proceso.at[index, 'estado'] = '4.2 Grandes cambios'

        # if decision_editor == Pequeños cambios / Minor revision (los textos no son estos)
            # estado -> 4.1 Pequeños cambios (los textos no son estos)
        elif row['decision_editor_def'] == 'Pequeños cambios / Minor revision':
            new_manuscritos_en_proceso.at[index, 'estado'] = '4.1 Pequeños cambios'



In [9]:
last_row

990

In [10]:
#change the rows numbers of new_manuscritos_en_proceso to start from the last row number of old_data + 1
new_manuscritos_en_proceso.index = range(last_row +1, last_row +1+len(new_manuscritos_en_proceso))

In [11]:
#print the fields referencia, revision, año, fecha_alta from new_manuscritos_en_proceso
print (new_manuscritos_en_proceso[['numero_manuscrito','referencia', 'revision', 'año', 'fecha_alta', 'entrada', 'decision_editor_r1', 'estado', 'responsable']])

    numero_manuscrito        referencia revision     año  fecha_alta  \
991  NEFRO-D-23-00040  NEFRO-D-23-00040           2023.0  31/01/2023   
992  NEFRO-D-23-00039  NEFRO-D-23-00039           2023.0  31/01/2023   
993  NEFRO-D-23-00013  NEFRO-D-23-00013           2023.0  31/01/2023   
994  NEFRO-D-23-00012  NEFRO-D-23-00012           2023.0  31/01/2023   

        entrada decision_editor_r1                                 estado  \
991  27/01/2023                NaN  1. Nuevo esperando revisor o rechazar   
992  27/01/2023                NaN  1. Nuevo esperando revisor o rechazar   
993  12/01/2023         12/01/2023                    4.2 Grandes cambios   
994  12/01/2023         12/01/2023                    4.2 Grandes cambios   

           responsable  
991  Mariano / Roberto  
992  Mariano / Roberto  
993                NaN  
994                NaN  


In [12]:
#old_data_bak = old_data.copy()
#old_data = old_data_bak.copy()

In [13]:
old_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 126 entries, 14 to 764
Data columns (total 63 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   data_identity               0 non-null      float64
 1   numero_manuscrito           126 non-null    object 
 2   referencia                  126 non-null    object 
 3   revision                    0 non-null      float64
 4   titulo                      126 non-null    object 
 5   tipo_articulo               0 non-null      float64
 6   autor                       0 non-null      float64
 7   email                       0 non-null      float64
 8   fecha_inicial               0 non-null      float64
 9   estado_actual               0 non-null      float64
 10  fecha_estado                0 non-null      float64
 11  fecha_vencimiento           0 non-null      float64
 12  editor                      0 non-null      float64
 13  decision_editor_def         0 non-

In [14]:
#add the data merging the new_manuscritos_en_proceso dataframe with the old_data dataframe
old_data = pd.concat([old_data, new_manuscritos_en_proceso], sort=False)


#print the new data in old_data dataframe. are the rows with referencia == NEFRO-D-23-00040, NEFRO-D-23-00039, NEFRO-D-23-00013, NEFRO-D-23-00012
old_data.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 130 entries, 14 to 994
Data columns (total 63 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   data_identity               4 non-null      float64
 1   numero_manuscrito           130 non-null    object 
 2   referencia                  130 non-null    object 
 3   revision                    4 non-null      object 
 4   titulo                      130 non-null    object 
 5   tipo_articulo               4 non-null      object 
 6   autor                       4 non-null      object 
 7   email                       2 non-null      object 
 8   fecha_inicial               4 non-null      object 
 9   estado_actual               4 non-null      object 
 10  fecha_estado                4 non-null      object 
 11  fecha_vencimiento           2 non-null      object 
 12  editor                      2 non-null      object 
 13  decision_editor_def         2 non-

/var/folders/ch/63kj7hh97f1_0yrh4d1t0t8c0000gn/T/ipykernel_15476/975985572.py:2: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  old_data = pd.concat([old_data, new_manuscritos_en_proceso], sort=False)


# Detectar las bajas de manuscritos

Está en old_data sin fecha de baja pero no están en en_proceso_actual, lo que acabamos de descargar. 
Busco el id en definitiva_actual para ver cual ha sido el status_final
Añado la fecha de hoy como fecha de baja en definitiva_guardados

In [17]:
#find the manuscritos that are in old_data but not in new_manuscritos_en_proceso

borrados = old_data[~old_data['referencia'].isin(en_proceso['numero_manuscrito'])]

print (len(borrados), 'manuscritos borrados')
print (borrados[['referencia', 'revision', 'año', 'fecha_alta', 'entrada', 'decision_editor_r1', 'estado', 'responsable']])

76 manuscritos borrados
           referencia revision     año fecha_alta     entrada  \
14   NEFRO-D-21-00104      NaN  2021.0        NaN  2021-03-02   
16   NEFRO-D-22-00086      NaN  2022.0        NaN  2022-03-07   
18   NEFRO-D-22-00159      NaN  2022.0        NaN  2022-04-24   
27   NEFRO-D-23-00027      NaN  2023.0        NaN  2023-01-19   
52   NEFRO-D-23-00001      NaN  2023.0        NaN  2023-01-02   
..                ...      ...     ...        ...         ...   
466  NEFRO-D-22-00024      NaN  2022.0        NaN  2022-01-19   
486  NEFRO-D-22-00005      NaN  2022.0        NaN  2022-01-04   
495  NEFRO-D-21-00587      NaN  2021.0        NaN  2021-12-27   
544  NEFRO-D-21-00534      NaN  2021.0        NaN  2021-11-23   
764  NEFRO-D-21-00293      NaN  2021.0        NaN  2021-06-28   

    decision_editor_r1                                 estado      responsable  
14          2021-05-14                    4.2 Grandes cambios          Autores  
16          2022-05-11           

In [19]:
len(old_data)

130